## Quantum Fourier Transform (QFT)

The Quantum Fourier Transform (QFT) is a core component of many quantum algorithms. It transforms a basis state into a superposition of all possible states, each with a distinct phase. This makes it a powerful tool for uncovering hidden patterns, periodic repetitions, or unknown phases. The standard implementation of the QFT is based on the Coppersmith formula [1]:

$$
|x_{n-1}\dots x_0\rangle \;\mapsto\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_0}|1\rangle\Big)\;\otimes\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_1x_0}|1\rangle\Big)
\;\otimes\;\cdots\;\otimes\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_{n-1}x_{n-2}\dots x_0}|1\rangle\Big).
$$
where $0.x_j x_{j-1} \dots x_0$ denotes the binary fraction. For example,

$$
0.x_2 x_1 x_0 = \frac{x_2}{2} + \frac{x_1}{4} + \frac{x_0}{8}.
$$

Here, the first qubit factor depends on the **least significant bit** $x_{0}$, and each subsequent factor accumulates more bits from right to left.

However, in the quantum computing literature, the more common textbook interpretation is given by Nielsen & Chuang [2]. In this interpretation, the first qubit factor depends on the **most significant bit** $x_{n-1}$, and each subsequent factor accumulates more bits from left to right:

$$
|x_{n-1}\dots x_0\rangle \;\mapsto\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_{n-1}}|1\rangle\Big)\;\otimes\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_{n-2}x_{n-1}}|1\rangle\Big)
\;\otimes\;\cdots\;\otimes\;
\frac{1}{\sqrt{2}}\Big(|0\rangle + e^{2\pi i \, 0.x_0x_1\dots x_{n-1}}|1\rangle\Big).
$$

Each term in these formulas represents a single qubit prepared in a superposition of the form $ \frac{1}{\sqrt{2}}\left(|0\rangle + e^{i\varphi}|1\rangle\right) $. The relative phase $e^{i\varphi}$ of the $|1\rangle$ component is determined by a binary fraction.  As we move from the most significant to the least significant qubit (in the Nielsen & Chuang convention), each subsequent qubit carries a phase shift with finer resolution, differing by multiples of $1/2^k$.  Thus, the terms are written in order from the largest to the smallest phase contributions.  
Consequently, on each qubit line, the construction applies an H gate followed by a sequence of controlled phase gates ($P$ gates).  As we move upward through the circuit, each qubit receives one more $P$ gate than the one below it, corresponding to the binary digits that control the accumulated phase.

In both conventions, the qubits end up in **reversed order** compared to the canonical Fourier basis. Therefore, **a sequence of SWAP gates** is usually added at the end of the QFT circuit to restore the correct ordering of qubits.

<br>
<div align="center">
<img src="images/QFT-3-qubits.png"><br>
<em>Figure: 3-qubit Quantum Fourier Transform circuit (textbook style)</em>
</div> 

The inverse QFT (IQFT) is obtained by reversing the QFT circuit, inverting each controlled phase, and swapping the qubits back into their original order.  

### Task

- Implement a `qft` function that creates an n-qubit circuit following the textbook (Nielsen & Chuang) style.
- Implement the inverse Quantum Fourier Transform circuit `iqft`, based on the previous function.  
- Demonstrate a QFT → IQFT round-trip on at least one basis state and report counts concentrating on the original bitstring.

### Expected Output

- Drawn circuits for the QFT and the inverse QFT.  
- A demonstration of the QFT → IQFT round-trip on a chosen basis state.  
- Measurement results showing counts concentrated on the original bitstring, confirming correctness of the implementation.

### Experimentation

- Test the `qft` function with different numbers of qubits and compare the circuit depth and structure in textbook versus bottom-up style.  
- Apply the QFT to various input states (e.g., computational basis states or superposition states) and observe the resulting measurement distributions.  
- Explore how the presence or absence of the final SWAP gates affects the ordering of the output states.  
- Verify the correctness of the `iqft` by composing QFT → IQFT for different inputs and confirming recovery of the original state.  
- Implement a `qft` function that creates an n-qubit circuit following the **Coppersmith (bottom-up)** style.

---

References: 

[1] D. Coppersmith, *An approximate Fourier transform useful in quantum factoring*, IBM Research Report RC19642, 1994.

[2] M. A. Nielsen and I. L. Chuang, *Quantum Computation and Quantum Information*, 10th Anniversary ed. Cambridge, U.K.: Cambridge Univ. Press, 2010.


In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import numpy as np


def qft(n, insert_barriers = True):
    """
    Quantum Fourier Transform (QFT) textbook style:target goes top -> bottom

    Args:
        n: number of qubits
        insert_barriers: keep textbook layering in the drawing
    """
    qc = QuantumCircuit(n)
    for j in range(n):
        qc.h(j)
        # controls are below the target: j+1..n-1
        for k in range(j + 1, n):
            theta = np.pi / (2 ** (k - j))  # π/2, π/4, ...
            qc.cp(theta, k, j)
        if insert_barriers:
            qc.barrier()
    
    for i in range(n // 2):
        qc.swap(i, n - 1 - i)
        
    return qc

def iqft(n, insert_barriers = True):
    """Inverse QFT in the same convention."""
    return qft(n, insert_barriers=insert_barriers).inverse()

# -----------------------------------------------
#                main program
# -----------------------------------------------

n = 4
print("Textbook QFT layout:")
qc_t = qft(n)
display(circuit_drawer(qc_t, output="mpl"))

print("Textbook Inverse QFT layout:")
iqc_t = iqft(n)
display(circuit_drawer(iqc_t, output="mpl"))

# -- test the circuits --
bitstring = "1101" # basis state to test

qc = QuantumCircuit(n)
for i, bit in enumerate(reversed(bitstring)): # prepare the input state
    if bit == "1":
        qc.x(i)   # flip qubit i to |1>
                
qc.compose(qc_t, inplace=True)
qc.compose(iqc_t, inplace=True)
qc.measure_all()

# --- Simulate and show results ---
simulator = AerSimulator()
result = simulator.run(qc, shots=2048).result()
counts = result.get_counts()

print(f"Testing round-trip on |{bitstring}⟩ with exact QFT → iQFT: {counts}")
